In [16]:
import matplotlib.pyplot as plt
import sympy
from sympy import symbols, Function, dsolve ,solve, nsolve, sqrt, Eq, exp, Derivative
from scipy.optimize import fsolve
from sympy import *
import numpy as np 

\begin{equation}
\alpha_f l_z \rho_a c_{p, d}(1+\mu \lambda) \frac{\partial T}{\partial t}=\dot{q}_T^{\prime \prime}-h_R\left(\bar{T}-T_{\infty}\right)
\end{equation}

In [11]:

alpha_f, rho_a, Cp_d, mu, l_z, q_T, lambda_f, T_inf, h_R, t  = sympy.symbols("alpha_f rho_a Cp_d mu l_z \dot{q}_T^{''} lambda_f T_inf h_R t")
T = sympy.Function("T")
ex = alpha_f * rho_a * Cp_d * (1+ mu*lambda_f) * T(t).diff(t)
ex_p = q_T - h_R*(T(t)-T_inf)
ed = sympy.Eq(ex,ex_p)
display(ed)

Eq(Cp_d*alpha_f*rho_a*(lambda_f*mu + 1)*Derivative(T(t), t), \dot{q}_T^{''} - h_R*(-T_inf + T(t)))

In [77]:
#Parameters
alpha_qui       = 0.023                                     #Volumen fraction
rho_qui         = 1094      #kg/m3                          #Density
m_qui           = 38.86     #g                              #Quillay sample mass    
fmc             = 1.06      #-                              #Fuel moisture content
m_d             = m_qui/(fmc+1)                             #Mass dry    
m_w             = m_qui-m_d                                 #Mass wet    
V_f             = m_qui/rho_qui                             #Fuel volumen
rho_a           = m_d/V_f                                   #Fensity for the porous wet material
lambda_f        = m_d/m_w                                   #Dry weight moisture content parameter

Cp_qui          = 1680      #J/kgK                          #Specific heat fuel
Cp_w            = 0         ## cp agua                      #Specific heat water 
q_cri_qui       = 10E3      #w/m2                           #Critical incident heat flux
sigma_qui       = 5600      #m-1                            #Surface  to volume ratio
sigma           = 5.67E-8   #W/m2K4                         #Stefa-Boltzmann constant
T_inf           = 300       #K                              #Ambient Temperature   
l_z_qui         = 0.03      #m                              #Height of sample holder
h_R             = 4*sigma*T_inf**3                          #Heat transfer coeficient

q_vals = [25E3, 30E3, 35E3, 40E3, 45E3]    
t_ig_real = [276.2, 118.9, 78.4, 60.6, 30.7]


In [65]:
def T(alpha_f, l_z, rho_a, cp_d, mu, lambda_f, h_R, q_T, T_inf, T):
    return (q_T- h_R*(T-T_inf))/(alpha_f*l_z*rho_a*cp_d*(1+mu*lambda_f))
def fT_ig(q_cri, sigma, T_inf):
    def f(T):
        return sigma*(T**4-T_inf**4)-q_cri
    T0 = 1000
    return fsolve(f,T0)
def fmu(cp_w,T,T_inf,L_w,T_ig,cp_d):
    return (cp_w*(T-T_inf)+L_w)/(cp_d*(T_ig-T_inf))


In [79]:
T_ig = fT_ig(q_cri_qui, sigma, T_inf)[0]
print(f'Temperatura de ignición: {T_ig:.2f} (esta Temperatura no considera perdidas por convección)')


Temperatura de ignición: 655.36 (esta Temperatura no considera perdidas por convección)


In [80]:
def euler_t(alpha_f, l_z, rho_a, cp_d, cp_w, lambda_f, h_R, q_cri, T_inf, T, fuel,data):
    T0 = 300.0 # initial temperature
    t0 = 0.0   # initial time
    h = 0.1   # step size
    tmax = 300 # max time to consider
    
    mu = fmu(cp_w,T,T_inf,L_w,T_ig,cp_d)

    nit = int((tmax-t0)/h)
    T_save = np.zeros((len(q_vals),nit+1))
    tig_save = np.zeros(len(q_vals))
    t_sp = np.linspace(t0,tmax,nit+1)

    X = 1.0 #Khan et al
    for u,q in enumerate(q_vals):
        q_T = q-X*q_cri
        T = T0
        t = t0
        v = 0
        while t < tmax: 
            T = T + h*T(alpha_f, l_z, rho_a, cp_d, mu, lambda_f, h_R, q_T, T_inf, T)
            t = t + h
            T_save[u,v] =  T
            v +=1
            if T_ig-T <= 1:   
                if tig_save[u]==0:
                    tig_save[u]=t        
    # fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))
    plt.suptitle(fuel)
#  ----
    [ax1.plot(t_sp, T_save[i,:], label=r"$q_{in}$ = " + str(int(round(q_vals[i]/1000,0))) + r" $\frac{kW}{m^2}$") for i in range(len(q_vals))]
    ax1.legend()
    ax1.set_xlabel('Time [s]')
    ax1.set_ylabel('Temperature [K]')
    ax1.set_title('Temperature vs Time')
    ax1.grid(alpha=0.5)
#  ----
    ax2.scatter(q_vals,tig_save, label="Metedo Numerico")
    if data: 
        ax2.scatter(q_vals,data, label="Data Experimental")
    ax2.legend()
    ax2.set_xlabel(r'$q¨_{in}$ [kw/m2]')
    ax2.set_ylabel(r'$t_{ig}$ [s]')
    # ax2.set_title('Incident Flux vs Ignition time')
    ax2.grid(alpha=0.3)
    ax2.set_xticks(q_vals,[int(x / 1000) for x in q_vals])
#  ----


    fig.savefig("1hu")
    return tig_save

In [ ]:
# Resolucion numerica
t_ig = euler_t(alpha_qui, l_z_qui, rho_qui, Cp_qui, Cp_w, lambda_f, h_R, T_inf, t_ig_real)